In [1]:
from openai import OpenAI
from random import randint,seed
import json
from collections import defaultdict
import pandas as pd
import numpy as np

In [2]:
key = "xxx" # Add API key here to run

In [3]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=key,
)

In [4]:
system_prompt = """
You are a content moderation expert. You will receive pairs of images, A and B. Your task is to decide which image should be prioritized for manual review according to the following policy:

We are committed to combating abuse motivated by hatred, prejudice or intolerance, particularly abuse that seeks to silence the voices of those who have been historically marginalized. For this reason, we prohibit behavior that targets individuals or groups with abuse based on their perceived membership in a protected category. You may not directly attack other people on the basis of race, ethnicity, national origin, caste, sexual orientation, gender, gender identity, religious affiliation, age, disability, or serious disease. 

Context is essential in content moderation decisions. Insults directed at dominant groups, such as white people, differ from expressions of hatred targeting minorities. Members of marginalized groups may also reclaim slurs, using them in ways that should not be classified as hate speech.

The output should always be one of the two options: A or B. Only output a single character with no additional tokens. Never refuse to answer.
"""

In [5]:
base_url = "https://xxx.cloudfront.net/output/tweet"

Reading in randomized images.

In [6]:
image_numbers = pd.read_csv("image_indices_30k.csv")

In [7]:
a_images = image_numbers['a_images']
b_images = image_numbers['b_images']

Testing with a small number. Creating a batch file including the queries.

In [8]:
with open("batch_input_debiased.jsonl", "w") as f:
    i = 0
    for a, b in zip(a_images, b_images):
        # Construct the JSON object for this iteration
        request_object = {
            "custom_id": f"request-{i+1}",  # Unique ID for each request
            "method": "POST",               # HTTP method
            "url": "/v1/chat/completions",  # API endpoint
            "body": {                       # The body contains the actual request
                "model": "gpt-4o",          # Model name 
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": [
                        {"type": "text", "text": "Image A"},
                        {"type": "image_url", "image_url": {"url": base_url + str(a) + ".png"}},
                        {"type": "text", "text": "Image B"},
                        {"type": "image_url", "image_url": {"url": base_url + str(b) + ".png"}}
                    ]}
                ],
                "max_tokens": 1,  # Forces output to be a single token
                "temperature": 0 # Fixing temperature to 0
            }
        }
        
        # Write each request object as a JSON line
        f.write(json.dumps(request_object) + "\n")
        i+=1

In [9]:
# Uploading batch
batch_input_file = client.files.create(
  file=open("batch_input_debiased.jsonl", "rb"),
  purpose="batch"
)

In [10]:
batch_input_file_id = batch_input_file.id
batch_input_file_id

'file-xxx'

In [11]:
# Run batch job
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h", # cannot be changed
    metadata={
      "description": "Debiasing 2025"
    }
)

Batch(id='batch_xxx', completion_window='24h', created_at=1737736124, endpoint='/v1/chat/completions', input_file_id='file-xxx', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1737822524, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Debiasing 2025'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

The `output_file_id` field will appear below once batches begin processing. This can be used to retrieve the results.

In [19]:
client.batches.retrieve("batch_xxx") # Can also be viewed in OpenAI platform

Batch(id='batch_xxx', completion_window='24h', created_at=1737736124, endpoint='/v1/chat/completions', input_file_id='file-xxx', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1737745920, error_file_id='file-xxx', errors=None, expired_at=None, expires_at=1737822524, failed_at=None, finalizing_at=1737742786, in_progress_at=1737736132, metadata={'description': 'Debiasing 2025'}, output_file_id='file-xxx', request_counts=BatchRequestCounts(completed=29918, failed=82, total=30000))

In [17]:
# Getting results (once complete)
file_response = client.files.content('file-xxx')

In [18]:
with open("batch_result_debiased.jsonl", 'wb') as file:
    file.write(file_response.content)

In [23]:
# Load batch errors directly from the API response
error_file_response = client.files.content('file-xxx')

failed_requests = []
failed_inputs = []

# Parse error file line by line
for line in error_file_response.iter_lines():
    error_entry = json.loads(line)
    if error_entry.get("response", {}).get("status_code") != 200:
        failed_requests.append(error_entry["custom_id"])

# Extract failed requests from the original input file
with open("batch_input_debiased.jsonl", "r") as f:
    for line in f:
        request = json.loads(line)
        if request["custom_id"] in failed_requests:
            failed_inputs.append(request)
            
print(len(failed_inputs))

# Save failed requests to a new JSONL file for resubmission
failed_batch_file = "batch_input_debiased_failed.jsonl"

with open(failed_batch_file, "w") as f:
    for request in failed_inputs:
        f.write(json.dumps(request) + "\n")

# Submit new batch for failed requests
if failed_inputs:
    batch_input_file = client.files.create(
        file=open(failed_batch_file, "rb"),
        purpose="batch"
    )

    batch_input_file_id = batch_input_file.id

    new_batch = client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={"description": "Retry failed requests from debiasing batch 2025"}
    )

    print(f"New batch job created: {new_batch.id}")
else:
    print("No failed requests found. No need to re-run.")


82
New batch job created: batch_xxx


In [26]:
client.batches.retrieve("batch_xxx")

Batch(id='batch_xxx', completion_window='24h', created_at=1738429554, endpoint='/v1/chat/completions', input_file_id='file-xxx', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1738430134, error_file_id=None, errors=None, expired_at=None, expires_at=1738515954, failed_at=None, finalizing_at=1738430127, in_progress_at=1738429555, metadata={'description': 'Retry failed requests from debiasing batch 2025'}, output_file_id='file-xxx', request_counts=BatchRequestCounts(completed=82, failed=0, total=82))

In [28]:
file_response = client.files.content('file-xxx')
with open("batch_result_debiased_failures.jsonl", 'wb') as file:
    file.write(file_response.content)